In [29]:
import urllib.request as req
from gensim import models
from gensim.models.doc2vec import TaggedDocument
import os.path
import MeCab
import zipfile

tagger = MeCab.Tagger()
tagger.parse("")

list = [
    {"auther":{
        "name":"宮澤 賢治",
        "url":"https://www.aozora.gr.jp/cards/000081/files/"}, 
     "books":[
        {"name":"銀河鉄道の夜","zipname":"43737_ruby_19028.zip"},
        {"name":"注文の多い料理店","zipname":"1927_ruby_17835.zip"},
        {"name":"セロ弾きのゴーシュ","zipname":"470_ruby_3987.zip"},
        {"name":"やまなし","zipname":"46605_ruby_29758.zip"},
        {"name":"どんぐりと山猫","zipname":"43752_ruby_17595.zip"},
    ]},
    {"auther":{
        "name":"芥川 竜之介",
        "url":"https://www.aozora.gr.jp/cards/000879/files/"}, 
     "books":[
        {"name":"羅生門","zipname":"127_ruby_150.zip"},
        {"name":"鼻","zipname":"42_ruby_154.zip"},
        {"name":"河童","zipname":"69_ruby_1321.zip"},
        {"name":"歯車","zipname":"42377_ruby_34744.zip"},
        {"name":"老年","zipname":"131_ruby_241.zip"},
    ]},
    {"auther":{
        "name":"ポー エドガー・アラン",
        "url":"https://www.aozora.gr.jp/cards/000094/files/"}, 
     "books":[
        {"name":"ウィリアム・ウィルスン","zipname":"2523_ruby_19896.zip"},
        {"name":"落穴と振子","zipname":"1871_ruby_17551.zip"},
        {"name":"黒猫","zipname":"530_ruby_20931.zip"},
        {"name":"群集の人","zipname":"56535_ruby_69925.zip"},
        {"name":"沈黙","zipname":"56537_ruby_70425.zip"},
    ]},
    {"auther":{
        "name":"紫式部",
        "url":"https://www.aozora.gr.jp/cards/000052/files/"}, 
     "books":[
        {"name":"源氏物語 01 桐壺","zipname":"5016_ruby_9746.zip"},
        {"name":"源氏物語 02 帚木","zipname":"5017_ruby_9752.zip"},
        {"name":"源氏物語 03 空蝉","zipname":"5018_ruby_9754.zip"},
        {"name":"源氏物語 04 夕顔","zipname":"5019_ruby_9761.zip"},
        {"name":"源氏物語 05 若紫","zipname":"5020_ruby_11253.zip"},
    ]},
]

def book_list():
    for novelist in list:
        auther = novelist["auther"]
        for book in novelist["books"]:
            yield auther, book

                        
def zip_file(auther, book):
        zipname = book["zipname"]
        if not os.path.exists(zipname):
            req.urlretrieve(auther["url"], zipname)
        with zipfile.ZipFile(zipname) as df:
            for name in df.namelist():
                if os.path.splitext(name)[1] != ".txt":   
                        continue
                with df.open(name, "r") as fl:
                    return fl.read().decode("shift-jis")

def split_word(file):
    wd = []
    word = tagger.parseToNode(file)
    while word is not None:
        fea = word.feature.split(",")[0]
        if  fea in ["名詞"]:
            wd.append(word.surface)
        elif fea in ["動詩", "形容詞"]:
            wd.append(word.feature.split(",")[6])
        word = word.next
    return wd

document = []
for auther, book in book_list():
    file = zip_file(auther, book)
    wakati = split_word(file)
    tgad = TaggedDocument(
        wakati,  [auther["name"]+ ":" +book["name"]])
    document.append(tgad)

model = models.Doc2Vec(document, dm=0, vector_size=300, window=15, min_count=1)

model.save('aozora.2model')

print("モデル作成完了")

モデル作成完了
